# Starfinder: Soldier with Fangblade

Consider a {srd_classes}`Soldier<5-soldier>` (Close Quarters) with a {srd_weapons}`Fangblade <14-fangblade>` (d10 Backswing, Boost d12).

Which attack routine is better?

1. Primary Target, Area Fire, Strike (don't use the Boost trait)
2. Boost, Primary Target, Area Fire (Boost damage goes to the Primary Target Strike)
3. Boost, Area Fire (deliberately skip the Primary Target Strike to deal more damage on Area Fire)
4. *Benchmark:* a max-level fireball from a dedicated spellcaster

In [ ]:
# Install in jupyterlite
%pip install -q pathfinder2e-stats

import numpy as np
import xarray

import pathfinder2e_stats as pf2

In [ ]:
level = 5
whirling_swipe = True  # +1 to area fire DC for Backswing and Swipe weapons

atk = (
    pf2.tables.SIMPLE_PC.weapon_attack_bonus.soldier.sum("component")
    .sel(level=level)
    .item()
)
area_fire_DC = (
    pf2.tables.SIMPLE_PC.area_fire_DC.soldier.sum("component").sel(level=level).item()
)
fireball_DC = (
    pf2.tables.SIMPLE_PC.spell_DC.wizard.sum("component").sel(level=level).item()
)

print(f"{atk=}, {area_fire_DC=}, {fireball_DC=}")

In [ ]:
weapon_dice = pf2.tables.PC.weapon_dice.improvement.sel(level=level).item()
weapon_specialization = pf2.tables.PC.weapon_specialization.soldier.sel(
    level=level
).item()
strength = pf2.tables.PC.ability_bonus.boosts.sel(initial=3).sel(level=level).item()

fangblade = pf2.armory.starfinder.melee.fangblade(
    weapon_dice, strength + weapon_specialization
)
fangblade

In [ ]:
fireball = pf2.armory.spells.fireball(rank=pf2.level2rank(level))
fireball

In [ ]:
enemy = pf2.tables.SIMPLE_NPC.sel(level=level, drop=True)[
    ["AC", "saving_throws", "HP"]
] * xarray.DataArray(
    np.ones((3, 4), dtype=int),
    dims=["target", "routine"],
    coords={
        "target": ["primary", "secondary 1", "secondary 2"],
        "routine": ["no_boost", "boost_primary", "boost_area", "fireball"],
    },
)
enemy.isel(target=0, routine=0, drop=True).display()

In [ ]:
# Primary and secondary targets use the same damage roll, but save independently.
# 'challenge' and 'routine' are what-if analyses - let's compare the same dice
# rolls in different situations.
pf2.set_config(
    check_dependent_dims=("challenge", "routine"),
    check_independent_dims=("target",),
    damage_dependent_dims=("challenge", "routine", "target"),
)

In [ ]:
strike1 = pf2.check(atk, DC=enemy.AC)
strike1["outcome"] = xarray.where(
    (strike1.target != "primary")
    | (strike1.routine == "boost_area")
    | (strike1.routine == "fireball"),
    pf2.DoS.no_roll,
    strike1.outcome,
)
strike1 = xarray.concat(
    [
        pf2.damage(strike1.sel(routine="no_boost"), fangblade.apply_boost(False)),
        pf2.damage(strike1.sel(routine="boost_primary"), fangblade.apply_boost(True)),
        pf2.damage(strike1.sel(routine="boost_area"), pf2.Damage("slashing", 0, 0)),
        pf2.damage(strike1.sel(routine="fireball"), pf2.Damage("fire", 0, 0)),
    ],
    dim="routine",
    join="outer",
)
strike1.total_damage.mean("roll").stack(col=["challenge", "routine"]).display()

In [ ]:
ws_bonus = 1 if whirling_swipe else 0
DC = xarray.DataArray(
    [
        area_fire_DC + ws_bonus,
        area_fire_DC + ws_bonus,
        area_fire_DC + ws_bonus,
        fireball_DC,
    ],
    dims=["routine"],
)

area_fire_check = pf2.check(enemy.saving_throws, DC=DC, primary_target=strike1)

fangblade_area = fangblade.area_fire()
area_fire = xarray.concat(
    [
        pf2.damage(
            area_fire_check.sel(routine="no_boost"), fangblade_area.apply_boost(False)
        ),
        pf2.damage(
            area_fire_check.sel(routine="boost_primary"),
            fangblade_area.apply_boost(False),
        ),
        pf2.damage(
            area_fire_check.sel(routine="boost_area"), fangblade_area.apply_boost(True)
        ),
        pf2.damage(area_fire_check.sel(routine="fireball"), fireball),
    ],
    dim="routine",
    join="outer",
)
area_fire.total_damage.mean("roll").stack(col=["challenge", "routine"]).display()

**Axes critical specialization:** deal damage equal to the weapon damage dice to an adjacent target.
Conveniently, for 2+ targets in a 5-foot burst, all targets are adjacent by definition.

Note that Soldiers only get critical specialization in melee weapons when they use them to Area Fire.

For the sake of simplicity, we'll account for this damage on the target that received the critical hit instead of the one adjacent. We will remove it later for the use case when there is only one target.

In [ ]:
fangblade_area_crit = pf2.armory.critical_specialization.axe(
    fangblade.apply_boost(False)
).area_fire()
fangblade_area_crit

In [ ]:
area_crit = xarray.concat(
    [
        pf2.damage(
            area_fire_check.sel(routine=~(area_fire.routine == "fireball")),
            fangblade_area_crit,
        ),
        area_fire_check.sel(routine="fireball"),
    ],
    dim="routine",
    join="outer",
    data_vars="all",
).fillna(0)

area_crit.total_damage.mean("roll").stack(col=["challenge", "routine"]).display()

In [ ]:
# Note: Primary Target does not increase MAP, but Area Fire does
backswing = xarray.where(strike1.outcome < pf2.DoS.success, 1, 0)
strike2 = pf2.check(atk - 5 + backswing, DC=enemy.AC)
strike2["outcome"] = xarray.where(
    (strike2.target != "primary") | (strike2.routine != "no_boost"),
    pf2.DoS.no_roll,
    strike2.outcome,
)
strike2 = pf2.damage(strike2, fangblade.apply_boost(False))
strike2.total_damage.mean("roll").stack(col=["challenge", "routine"]).display()

In [ ]:
full_round = xarray.concat(
    [strike1, area_fire, area_crit, strike2],
    dim="action",
    join="outer",
)
full_round["action"] = ["primary_target", "area_fire", "area_crit", "strike"]

## Mean damage

In [ ]:
total_damage = full_round.total_damage.mean("roll")
total_damage = xarray.concat(
    [total_damage, total_damage.sum("action").expand_dims(action=["TOTAL"])],
    dim="action",
)
total_damage.stack(row=["target", "action"], col=["challenge", "routine"]).to_pandas().T

In [ ]:
grand_total = (
    full_round.total_damage.mean("roll")
    * xarray.DataArray(
        [[1, 0, 0], [1, 1, 0], [1, 1, 1]],
        dims=["# targets", "target"],
        coords={"# targets": [1, 2, 3]},
    )
).sum("target")

In [ ]:
_, crit_spec_on_one_target = xarray.align(
    grand_total,
    grand_total.sel({"# targets": [1], "action": ["area_crit"]}),
    join="outer",
    fill_value=0,
)

(grand_total - crit_spec_on_one_target).sum("action").stack(
    col=["challenge", "routine"]
).to_pandas().T

## Conclusions

The damage output of a Soldier with a Fangblade blows out of the water a fireball in all use cases.
The soldier can repeat their attack every round at no cost.
Additionally, when there are 3 or 4 targets, they have the flexibility to choose to concentrate the damage on the Primary Target (boost->primary target->area fire) or spread it equally (boost->area fire).

However, the fireball features a much larger area and only costs 2 actions instead of 3.